## A simple math problem

There is a group of 300 people who want to help each other. Unfortunately, a single person can meaningfully keep track of 150 people. Hovewer, all 300 want to cooperate and treat each other fairly. How can they do that?

It's sad that you will not find the problem in any schoolbook. If you did, the initial answer would be simple. Why don't they write down who helped whom and how much?

In [1]:
from dataclasses import dataclass, field
from time import time

@dataclass
class Transaction:
    recipient: str
    sender: str
    amount: float
    timestamp: float = field(default_factory=time)

## Key block

A block of transactions can help people keep track of any help provided. That way you don't need to remember how much help you gave and how much you received.

In [2]:
block = []
block.append(Transaction(
    sender='Alice', recipient='Bob', amount=2.0
))
block.append(Transaction(
    sender='Bob', recipient='Jon', amount=4.5
))

In [3]:
block

[Transaction(recipient='Bob', sender='Alice', amount=2.0, timestamp=1543083974.4465837),
 Transaction(recipient='Jon', sender='Bob', amount=4.5, timestamp=1543083974.4466345)]

This covers direct person to person cooperation. Everyone is happy but if someone tries to cooperate indirectly, things fall apart. Since anyone can update the block, transactions can be removed. If people try to exchange units of help, there is an incentive to mess with the transaction list.

In order to prevent tampering with transactions in a block we can hash the block and include the hash in the next block. If someone changes transactions, the hash will be different and the tampering will be apparent.

In [4]:
import hashlib

class Block:
    transactions: list
    previous_hash: str = 'coinbase'

    def __init__(self):
        self.transactions = []

    def append(self, t: Transaction):
        self.transactions.append(t)
    
    def __repr__(self):
        return 'transactions: {}\n\nprevious hash: {}'.format(self.transactions.__repr__(), self.previous_hash)
    
    def hash(self):
        return hashlib.sha256(str(self).encode()).hexdigest()
        

In [5]:
block = Block()
block.append(Transaction(
    sender='Alice', recipient='Bob', amount=2.0
))
block.append(Transaction(
    sender='Bob', recipient='Jon', amount=4.5
))

In [6]:
block

transactions: [Transaction(recipient='Bob', sender='Alice', amount=2.0, timestamp=1543083974.8359885), Transaction(recipient='Jon', sender='Bob', amount=4.5, timestamp=1543083974.8361387)]

previous hash: coinbase

In [7]:
block.hash()

'9b475a78e622b09eaa482300ef45e8f057f0ea05a0e03e8a7304834ff59ee0ca'

Hey, look at that! We got a short and unique (for our purposes) representation of our block and all the transactions in it.

In [8]:
block.append(Transaction(
    sender='Jon', recipient='Bob', amount=1.0
))

In [9]:
block.hash()

'ba899966c39498a80c9158dcae42cee1faba2e44f56c59b7981bd85766313068'

## Verification

Now anyone can put the blocks into a chain and it's easy to prove that no one tampered with prevous transactions.

In [10]:
import itertools


class Chain:
    blocks: list

    def __init__(self):
        """We need initial block"""
        block = Block()
        block.append(Transaction(
            sender='Alice', recipient='Alice', amount=2.0
        ))
        self.blocks = [block]

    def push(self, block: Block):
        block.previous_hash = self.blocks[-1].hash()
        self.blocks.append(block)

    @property
    def tampered(self):
        """
        This gives you the position of an invalid block according to its neighbour to the right.
        If the chain looks good, it returns 0.
        """
        a, b = itertools.tee(self.blocks)
        next(b, None)
        for position, pair in enumerate(zip(a, b)):
            if pair[0].hash() != pair[1].previous_hash:
                return position
        return -1

    @property
    def is_valid(self):
        if self.tampered < 0:
            return True
        return False
        

In [11]:
chain = Chain()

In [12]:
chain.blocks

[transactions: [Transaction(recipient='Alice', sender='Alice', amount=2.0, timestamp=1543083975.4887934)]
 
 previous hash: coinbase]

In [13]:
for _ in range(10):
    block = Block()
    for j in range(3):
        block.append(Transaction(sender='Bob', recipient='Jon', amount=j))
    chain.push(block)

In [14]:
chain.is_valid

True

In [15]:
borrowed = chain.blocks[0].transactions.pop()
borrowed

Transaction(recipient='Alice', sender='Alice', amount=2.0, timestamp=1543083975.4887934)

In [16]:
chain.is_valid

False

In [17]:
chain.tampered

0

Let's put the borrowed transaction back.

In [18]:
chain.blocks[0].transactions.append(borrowed)

In [19]:
chain.is_valid

True

We can only mess with transactions in the last block.

In [20]:
chain.blocks[-1].append(Transaction(recipient='Alice', sender='Jon', amount=5))

In [21]:
chain.is_valid

True

## Equal opportunity to verify

Now, who should have the right to create blocks and add them to the chain? Everyone who wants to! We just need to make sure it's not monopolized by a single person. Somehow we need to give an equal-ish chance for everyone to verify previous blocks and add new ones. Here comes the elegant part. What if we allow the previous hash field to only start with `00`? Our block will have a special `nonce` field that can have anything that makes our block hash start with `00`. If we use a good hashing algorithm, there is no other way but to try different possibilities until we get what we need. This kind of help work should be rewarded. We'll allow that person to assign help units to herself out of nothing.

In [22]:
class Block:
    transactions: list
    previous_hash: str = 'coinbase'
    nonce: str = 'blahblahblah'

    def __init__(self):
        self.transactions = []

    def append(self, t: Transaction):
        self.transactions.append(t)
    
    def __repr__(self):
        return 'transactions: {}\n\nprevious hash: {}\n\nnonce: {}'.format(
            self.transactions.__repr__(), self.previous_hash, self.nonce
        )
    
    def hash(self):
        return hashlib.sha256(str(self).encode()).hexdigest()
        

In [23]:
from uuid import uuid4

def add_block_with_proof_of_work(chain: Chain, block: Block):
    last_block = chain.blocks[-1]
    for i in range(100000):
        last_block.nonce = uuid4()
        temp_hash = last_block.hash()
        if temp_hash.startswith('00'):
            block.previous_hash = temp_hash
            chain.blocks.append(block)
            print('Found hash {} after {} iterations'.format(temp_hash, i))
            break
    else:
        print('After {} iterations, needed hash was not found.'.format(i))    

In [24]:
chain = Chain()

In [25]:
block = Block()
for j in range(3):
    block.append(Transaction(sender='Bob', recipient='Jon', amount=j))

In [26]:
add_block_with_proof_of_work(chain, block)

Found hash 00226307c0a1fee86375b96cbea86f25525f73ce63a0d5f21da8307ff032e9c5 after 71 iterations


In [27]:
chain.blocks

[transactions: [Transaction(recipient='Alice', sender='Alice', amount=2.0, timestamp=1543083976.9717913)]
 
 previous hash: coinbase
 
 nonce: e1c47d73-00d7-48dd-8069-fc2a39e8ca41,
 transactions: [Transaction(recipient='Jon', sender='Bob', amount=0, timestamp=1543083977.0724168), Transaction(recipient='Jon', sender='Bob', amount=1, timestamp=1543083977.0724344), Transaction(recipient='Jon', sender='Bob', amount=2, timestamp=1543083977.0724432)]
 
 previous hash: 00226307c0a1fee86375b96cbea86f25525f73ce63a0d5f21da8307ff032e9c5
 
 nonce: blahblahblah]

In [28]:
chain.is_valid

True

The facinating part is that we can have any number of arbitrary rules. As long as each helper (a.k.a miner) validates previous work, the system is stable. Each helper gets a reward if her block becomes part of the chain and loses that reward if someone else decides it's wrong. Since it's guaranteed that no one can validate their own work, everyone has to abide by an accepted set of rules.

## Asymmetric encryption to the rescue

There is one last major problem. Anyone can register a transaction from Alice (since she has a lot of help units) to Bob. It will be all valid but Alice might not even know she transferred her help units to Bob.

Sadly, Python doesn't have many encryption features in its standard library. Cryptography is a big field. There are lots of libs that can generate keys like PyCryptodome but we need something simple here. [ecdsa](https://github.com/warner/python-ecdsa) is the simplest to use for signature purposes. After `pip install ecdsa` we can import and use it.

In [29]:
import ecdsa
from base64 import b64decode, b64encode

sk = ecdsa.SigningKey.generate(curve=ecdsa.SECP256k1)
vk = sk.get_verifying_key()
signature = sk.sign(vk.to_string())
vk.verify(signature, vk.to_string())

True

The idea looks simple but `sk` (Signature Key) and `vk` (Verifying Key) are objects so we can't just put them into a transaction.

In [30]:
sk, vk

(<ecdsa.keys.SigningKey at 0x7fa7983fe080>,
 <ecdsa.keys.VerifyingKey at 0x7fa7983fe240>)

There are helper methods but it still looks ugly.

In [31]:
vk.to_string()

b'\xa4g_\xff\xd67B\x94Q\xca\x01\x17\xbfN\xf8\x8d\x18t\x0es\x01\x01\xabe\nU\x83\xb5\xda\x85\x14>H*\xc9\\a\xce\x9d\xbcy\xe8\x111\xdan>\xff=-)\xe3\xe3\xc4 \xbc~\x8d\\\xebXH\x00+'

In [32]:
signature

b'\x80\x8a\x9e\xcb\x14\x7f\x1bW\x7f\xe65\xbb\x11a_O\x95\\-Il\x11\xbd\xa9\xf2\xfd\xea\x86\x82\x01;\xb17\xba>sj\xb7\xea)^Y\xee\x8c\xb8\x88\x83F?\xad<q\\IW\xcb\xce\x1d\x15v\x0c\x10EB'

Let's encode them.

In [33]:
from base64 import b64encode, b64decode

In [34]:
public_key = b64encode(vk.to_string())
sign = b64encode(signature)

print('public key: {} \nsignature:  {}'.format(public_key, sign))

public key: b'pGdf/9Y3QpRRygEXv074jRh0DnMBAatlClWDtdqFFD5IKslcYc6dvHnoETHabj7/PS0p4+PEILx+jVzrWEgAKw==' 
signature:  b'gIqeyxR/G1d/5jW7EWFfT5VcLUlsEb2p8v3qhoIBO7E3uj5zarfqKV5Z7oy4iINGP608cVxJV8vOHRV2DBBFQg=='


That looks a little better. Are there any other possibilities?

In [35]:
from binascii import hexlify, unhexlify

In [36]:
public_key = hexlify(vk.to_string())
sign = hexlify(signature)

print('public key: {} \nsignature:  {}'.format(public_key, sign))

public key: b'a4675fffd637429451ca0117bf4ef88d18740e730101ab650a5583b5da85143e482ac95c61ce9dbc79e81131da6e3eff3d2d29e3e3c420bc7e8d5ceb5848002b' 
signature:  b'808a9ecb147f1b577fe635bb11615f4f955c2d496c11bda9f2fdea8682013bb137ba3e736ab7ea295e59ee8cb88883463fad3c715c4957cbce1d15760c104542'


It looks prettier but longer. I'll stick with the shorter version here. I always wondered why wallet addresses look so different for different coins. I guess the answer lies in encoding preferences. Onward to our signature field!

In [37]:
# Our new transaction data
@dataclass
class Transaction:
    recipient: bytes
    sender: bytes
    amount: float
    signature: bytes = b'' # New signature filed
    timestamp: float = field(default_factory=time)

In [38]:
def sign(public: bytes, private: bytes) -> bytes:
    pk = b64decode(private)
    signing_key = ecdsa.SigningKey.from_string(pk, curve=ecdsa.SECP256k1)
    signature = b64encode(signing_key.sign(public))
    return signature


def is_valid(transaction: Transaction) -> bool:
    pub_key = ecdsa.VerifyingKey.from_string(b64decode(transaction.sender))
    return pub_key.verify(b64decode(transaction.signature), transaction.sender)


class Person:
    public: bytes
    private: bytes

    def __init__(self):
        sk = ecdsa.SigningKey.generate(curve=ecdsa.SECP256k1)
        vk = sk.get_verifying_key()
        self.public = b64encode(vk.to_string())
        self.private = b64encode(sk.to_string())

    def __repr__(self):
        return str(self.public)

    def send_help(self, to:bytes, amount: float) -> Transaction:
        return Transaction(
            recipient=to,
            sender=self.public,
            signature=sign(self.public, self.private),
            amount=amount
        )

Now if Alice wants to send help to Bob, first she and Bob have to generate key pairs.

In [39]:
alice = Person()
bob = Person()
jon = Person()

In [40]:
alice.send_help(to=bob.public, amount=2.0)

Transaction(recipient=b'5f8ey9/bdKf//b7962U/SzHgyK/Uv2jXiH/3/AnnS68gZeMn2+fMvz/c/Ml/2N3RSlMUkC+PWSBcb0jCSm8BFQ==', sender=b'3AZNBuU+bGV5U59b1VRV/O86+J8xwR2fTswiRTuFzwIzC3dibhTfVPhHDq1Dk0Pb7pVmjH3Z2Ys/f5oYV4NJng==', amount=2.0, signature=b'ddT+0VAIEiyG4GZOS4yf+S6GE7aXoIK+JZk6zG5vw1CB4fENzh4jW0POAPLT5hKfnE50mK2TRCd320yFByFGxg==', timestamp=1543083979.0230663)

In [41]:
chain = Chain()

In [42]:
block = Block()

In [43]:
block.append(alice.send_help(to=bob.public, amount=10.0))
block.append(bob.send_help(to=jon.public, amount=5.0))
block.append(jon.send_help(to=alice.public, amount=2.5))

In [44]:
add_block_with_proof_of_work(chain, block)

Found hash 00c07f76ad355130f72b6bd738e5f3fe1ae7a4035f8dc430ae081fb8456e8748 after 112 iterations


In [45]:
chain.blocks

[transactions: [Transaction(recipient='Alice', sender='Alice', amount=2.0, signature=b'', timestamp=1543083979.051718)]
 
 previous hash: coinbase
 
 nonce: 14eafeac-0faf-4e9e-8ba5-97010f7e1901,
 transactions: [Transaction(recipient=b'5f8ey9/bdKf//b7962U/SzHgyK/Uv2jXiH/3/AnnS68gZeMn2+fMvz/c/Ml/2N3RSlMUkC+PWSBcb0jCSm8BFQ==', sender=b'3AZNBuU+bGV5U59b1VRV/O86+J8xwR2fTswiRTuFzwIzC3dibhTfVPhHDq1Dk0Pb7pVmjH3Z2Ys/f5oYV4NJng==', amount=10.0, signature=b'aKAzMzqJv4r3qCCNUtsvQ79xRRos4UYL7t8UGk0esjMMbsuJ+ivlAcTElJggXHqfV0XWrY9L9NgLB0XrOOIcPg==', timestamp=1543083979.3870409), Transaction(recipient=b'/AzV9soGtNgdb6CMdZBapbfIjpL1co46ih3ijDNlNVp98MCtH5pdZszrGr8afcg+/pg1muLz63RnES2D0L3K3A==', sender=b'5f8ey9/bdKf//b7962U/SzHgyK/Uv2jXiH/3/AnnS68gZeMn2+fMvz/c/Ml/2N3RSlMUkC+PWSBcb0jCSm8BFQ==', amount=5.0, signature=b'gHzSqG5iahDh7Bn9kQr/De3S7i5u+i8kp1NVTLRDH6BwNBbth7hOboh33bdyqBNDh87iWRH/CiaWuzZbcma0vg==', timestamp=1543083979.4903493), Transaction(recipient=b'3AZNBuU+bGV5U59b1VRV/O86+J8xwR2fTswiRTuFzw

In [46]:
block = Block()

In [47]:
block.append(alice.send_help(to=bob.public, amount=2.5))
block.append(bob.send_help(to=jon.public, amount=5.0))
block.append(jon.send_help(to=alice.public, amount=10.0))

In [48]:
add_block_with_proof_of_work(chain, block)

Found hash 00ad23d6c3d4a31a968e064eb84c96fcbf0e47329acf817cb99e58fb587fcf4b after 250 iterations


In [49]:
chain.blocks

[transactions: [Transaction(recipient='Alice', sender='Alice', amount=2.0, signature=b'', timestamp=1543083979.051718)]
 
 previous hash: coinbase
 
 nonce: 14eafeac-0faf-4e9e-8ba5-97010f7e1901,
 transactions: [Transaction(recipient=b'5f8ey9/bdKf//b7962U/SzHgyK/Uv2jXiH/3/AnnS68gZeMn2+fMvz/c/Ml/2N3RSlMUkC+PWSBcb0jCSm8BFQ==', sender=b'3AZNBuU+bGV5U59b1VRV/O86+J8xwR2fTswiRTuFzwIzC3dibhTfVPhHDq1Dk0Pb7pVmjH3Z2Ys/f5oYV4NJng==', amount=10.0, signature=b'aKAzMzqJv4r3qCCNUtsvQ79xRRos4UYL7t8UGk0esjMMbsuJ+ivlAcTElJggXHqfV0XWrY9L9NgLB0XrOOIcPg==', timestamp=1543083979.3870409), Transaction(recipient=b'/AzV9soGtNgdb6CMdZBapbfIjpL1co46ih3ijDNlNVp98MCtH5pdZszrGr8afcg+/pg1muLz63RnES2D0L3K3A==', sender=b'5f8ey9/bdKf//b7962U/SzHgyK/Uv2jXiH/3/AnnS68gZeMn2+fMvz/c/Ml/2N3RSlMUkC+PWSBcb0jCSm8BFQ==', amount=5.0, signature=b'gHzSqG5iahDh7Bn9kQr/De3S7i5u+i8kp1NVTLRDH6BwNBbth7hOboh33bdyqBNDh87iWRH/CiaWuzZbcma0vg==', timestamp=1543083979.4903493), Transaction(recipient=b'3AZNBuU+bGV5U59b1VRV/O86+J8xwR2fTswiRTuFzw

Okay, we sufficiently ridiculous-ified. However, it looks so scary only because of public keys and cryptographic signature. If we ignore those strings, things are actually very simple: 

1. There are transactions with who gave help to whom and a signature so there is a way to validate the transaction. 
2. There is a `nonce` field that we use to calculate hash for next block. 
3. There is a prevous hash field.

You probably noticed that anyone can send any amount of help units. How do we make sure that help units appear only from mining? We don't. We just make a rule that people can spend only as much as they have and only miners can assign value to themselves. As long as miners verify work of others, they will follow any kind of verifiable rules. In order to make sure a person doesn't spend more than she has, miners just need to sum up all the transactions involving that public address. A rule that transactions are valid only in rainy weather will not work because our chain doesn't contain this information.

## Is this all there is to blockchain and cryptocoins?

Yes, it covers main concepts. And no, I wouldn't trust more than a dollar to this particular one (even if we don't take into account lots of small missing parts). When I started this quest, I suspected that it's possible to uncover details that I wouldn't find just by reading articles about blockchains. The digest of what I learned while writing this simple blockchain is in [5 Things I learned from Implementing a Simple Blockchain](https://steemit.com/learning/@codeomatic/5-things-i-learned-from-implementing-a-simple-blockchain).